# Intro to LangChain

## Motivation
- LLM applications can be difficult to build and manage
- LangChain provides a framework for building modular components and chaining them together to produce an LLM app
- LangChain also provides plenty example apps and tools for connecting to various components (e.g. databases)
- LangServe and LangSmith are additional tools for exposing an LLM app as a REST API and testing/debugging/monitoring respectively

## Use Cases
- Document question answering
- Chatbots
- Analyzing structured data

## This Demo
We'll create a simple app that generates the correct SQL for a given question, given context on a database, using LangChain.

## Pre-requisites
- OpenAI account + API key
- Python 3, pip/pip3, jupyter notebook runtime

### Step One: Create LLM Objects

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

# Note: run `pip install -r requirements.txt` to configure packages. Latest openai version is not compatible with langchain yet so we use v0.28.1

# "OpenAI" and "ChatOpenAI" are essentially config objects (can specify temperature and other parameters)
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))
chat_model = ChatOpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
from langchain.schema import HumanMessage

text = "What SQL would you use to get the names of every employee?"
messages = [HumanMessage(content=text)]

print(llm.invoke(text))
print(chat_model.invoke(messages))



SELECT name FROM employees;
content='To get the names of every employee, you would use the following SQL query:\n\nSELECT name FROM employees;'


### Step Two: Create Prompt Templates

In [8]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate.from_template("Given a database with the following table schemas: {schemas}, give me the SQL for the following request: {request}")

### Step Three: Create Output Parser

In [9]:
from langchain.schema import BaseOutputParser

class LowercaseOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to all uppercase."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.upper()

print(LowercaseOutputParser().parse("select firstname from employees"))

SELECT FIRSTNAME FROM EMPLOYEES


### Step Four: Chain Them Together 

In [18]:
employeesSchema = """CREATE TABLE EMPLOYEES
      (EMPLOYEEID  CHAR(6)         NOT NULL,
       FIRSTNAME   VARCHAR(12)     NOT NULL,
       MIDINIT     CHAR(1)         NOT NULL,
       LASTNAME    VARCHAR(15)     NOT NULL,
       WORKDEPT    CHAR(3)                 ,
       PHONENO     CHAR(4)                 ,
       HIREDATE    DATE                    ,
       JOB         CHAR(8)                 ,
       EDLEVEL     SMALLINT        NOT NULL,
       SEX         CHAR(1)                 ,
       BIRTHDATE   DATE                    ,
       SALARY      DECIMAL(9,2)            ,
       BONUS       DECIMAL(9,2)            ,
       COMM        DECIMAL(9,2)         
       PRIMARY KEY (EMPLOYEEID))
"""

customersSchema = """CREATE TABLE CUSTOMERS
      (CUSTOMERID  VARCHAR(12)     NOT NULL,
       FIRSTNAME   VARCHAR(12)     NOT NULL,
       MIDINIT     CHAR(1)         NOT NULL,
       LASTNAME    VARCHAR(15)     NOT NULL,
       STOREID     VARCHAR(15)             ,
       PHONENO     VARCHAR(12)             ,
       ACTIVE      BOOLEAN                 ,
       PRIMARY KEY (CUSTOMERID))
"""

ordersSchema = """CREATE TABLE ORDERS
      (ORDERID     VARCHAR(12)      NOT NULL,
       CUSTOMERID  CHAR(12)                 ,
       PRIMARY KEY (ORDERID)                ,
       FOREIGN KEY (CUSTOMERID) REFERENCES CUSTOMERS(CUSTOMERID))        
"""

schemas = employeesSchema + customersSchema + ordersSchema

request = input("What is your request?")

chain = prompt | chat_model | LowercaseOutputParser()

# Example calls:
# find the storeid of a customer named "John A Smith"
# find all orders by a customer named "John A Smith"
# get the full name and salaries of employees in the sales department ordered by education level

print(chain.invoke({"schema": schemas, "request": request}))

TO FIND ALL ORDERS BY A CUSTOMER NAMED "JOHN A SMITH" FROM THE GIVEN DATABASE TABLES, YOU CAN USE THE FOLLOWING SQL QUERY:

SELECT O.ORDERID
FROM ORDERS O
JOIN CUSTOMERS C ON O.CUSTOMERID = C.CUSTOMERID
WHERE C.FIRSTNAME = 'JOHN' AND C.MIDINIT = 'A' AND C.LASTNAME = 'SMITH';


## Next Steps
- Currently, prompt takes in all table schemas in a database, which will cause problems at enterprise scale. Various solutions can be implemented to filter for relevant schemas; one such solution is using example selectors and running a similarity search on a vector store of example schemas to find relevant tables based on the request (see https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/similarity)
- Many prompt templates are provided in the LangChain community (example https://smith.langchain.com/hub/rlm/text-to-sql)
- Expose the app to consumers as a REST API with LangServe
- Test, debug, and monitor the app with LangSmith